In [10]:
#pip install nbformat

In [1]:
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as T
print(torch.__version__)

2.0.1


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

In [379]:
import string

all_characters= list(string.digits) + list(string.ascii_lowercase) 

print(all_characters)

nchar = len(all_characters)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [259]:
from pathlib import Path
current_directory = Path.cwd()

# Get the parent directory (one level up)
parent_directory = current_directory.parent
    
path_to_dataset =str(parent_directory)+'/captcha_dataset'

captchas = os.listdir(path_to_dataset)

X = np.zeros((len(captchas), 50, 200, 1))

y_strings = [] 

for i,img_name in enumerate(captchas):
    y_strings.append(img_name[:5])

    new_path = path_to_dataset + '/' + captchas[i]

    image = Image.open(new_path)

    array_image = np.array(image) / 255
    
    array_image = np.reshape(array_image[:,:,0], (50, 200, 1))

    X[i]=(array_image)

  

In [644]:
# make label array

y = np.zeros((5 , len(captchas) , nchar)) #5*1070*36
# digit * picture * digit label


for i,captcha in enumerate(y_strings):
   
    temp_label =np.zeros((5,nchar))

    for j, character in enumerate(captcha):
     
        character_index =  all_characters.index(character)

        temp_label[j,character_index] = 1
        
    y[:,i,:] = temp_label

In [646]:
class CaptchaModel(nn.Module):
    def __init__(self, nchar):
        super(CaptchaModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.mp1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.mp2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(32)
        self.mp3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        self.fc_shared = nn.Linear(4800, 64)
        self.fc_outputs = nn.ModuleList([nn.Linear(64, nchar) for _ in range(5)])

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.mp1(x)
        
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.mp2(x)
        
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.bn(x)
        x = self.mp3(x)
        
        x = x.view(x.size(0), -1)  # Flatten the tensor
        
        x = self.fc_shared(x)
        x = nn.functional.relu(x)
        
        outputs = [output_layer(x) for output_layer in self.fc_outputs]
        return outputs


model = CaptchaModel(nchar)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



In [647]:
from torch.utils.data import TensorDataset

# Convert your data and labels to PyTorch tensors
X_tensor = torch.FloatTensor(X.transpose(0, 3, 1, 2))
y_tensor = torch.LongTensor(y) 

# Split the data into training and validation sets
X_train = X_tensor[:round(1070*.8)]
y_train = y_tensor[:,:round(1070*.8),:]

X_val = X_tensor[round(1070*.8):]
y_val =y_tensor[:,round(1070*.8):,:]

train_dataset = TensorDataset(X_train, y_train[0], y_train[1], y_train[2], y_train[3], y_train[4])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

test_dataset = TensorDataset(X_val, y_val[0], y_val[1], y_val[2], y_val[3], y_val[4])
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [648]:
# Training loop
num_epochs = 100  # Number of training epochs
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    
    # Iterate over batches of the training dataset
    for inputs, target0, target1, target2, target3, target4 in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        
        # Forward pass
        outputs = model(inputs)
        
        # Calculate the loss for each output and sum them up
        targets = [target0, target1, target2, target3, target4]
        losses = [criterion(output, target.float()) for output, target in zip(outputs, targets)]
        loss = sum(losses)

        
        # Backpropagation
        loss.backward()
        
        # Update the model's parameters
        optimizer.step()
        
        running_loss += loss.item()
    
    # Print the average loss for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {running_loss / len(train_loader)}")

Epoch [1/100] - Loss: 17.090244999638312
Epoch [2/100] - Loss: 14.429562709949634
Epoch [3/100] - Loss: 12.572558579621491
Epoch [4/100] - Loss: 11.18979478765417
Epoch [5/100] - Loss: 10.028171821876809
Epoch [6/100] - Loss: 8.972186971593786
Epoch [7/100] - Loss: 8.081206074467412
Epoch [8/100] - Loss: 7.257201724582249
Epoch [9/100] - Loss: 6.641454573030825
Epoch [10/100] - Loss: 6.098852281217222
Epoch [11/100] - Loss: 5.454091584240949
Epoch [12/100] - Loss: 4.587676966631854
Epoch [13/100] - Loss: 3.9382301259923866
Epoch [14/100] - Loss: 3.530158457932649
Epoch [15/100] - Loss: 3.183662101074501
Epoch [16/100] - Loss: 2.853037401481911
Epoch [17/100] - Loss: 2.477067894405789
Epoch [18/100] - Loss: 2.099965965306317
Epoch [19/100] - Loss: 1.9440737830268011
Epoch [20/100] - Loss: 1.710588022514626
Epoch [21/100] - Loss: 1.3914876138722454
Epoch [22/100] - Loss: 1.2349813249376085
Epoch [23/100] - Loss: 1.0774433656975075
Epoch [24/100] - Loss: 0.9899191039579885
Epoch [25/100] 

In [542]:
# correct = 0
# total = 0

# with torch.no_grad():
#     model.eval()  # Set the model to evaluation mode
    
#     for inputs, target0, target1, target2, target3, target4 in test_loader:  # Assuming test_loader is your DataLoader
#         outputs = model(inputs)
        
#         # Calculate accuracy for each character
#         for output, target in zip(outputs, [target0, target1, target2, target3, target4]):
#             predicted = output.argmax(dim=1)  # Get the index of the highest predicted value
#            # predicted_labels[:, i] = predicted.numpy()
        
#             for i,j in enumerate(predicted):
                
#                 if target[i][j] == 1:
#                     correct += 1
#                 total += 1

# # Calculate overall accuracy
# accuracy = correct / total
# print(f"Accuracy: {accuracy:.2%}")



Accuracy: 5.79%


In [649]:
# Gets predicted labels from test dataset
pred_labels = torch.zeros((len(X_val),5,1))

with torch.no_grad():
    model.eval()



    for i,input in enumerate(X_val):
        
        temp_labels = torch.zeros((5,1))

        outputs = model(input.unsqueeze(0))

        for j,num in enumerate(outputs):
            #print(num.argmax(dim=1)[0])
            temp_labels[j] = num.argmax(dim=1)

        pred_labels[i] = temp_labels        

In [650]:
# turn labels into indexes to compare to predicted
y_test_indexes = torch.zeros((214,5,1))

for i in range(214):

    temp_label = torch.zeros((5,1))
    #print(y_val[:,i].size())

    for j in range(5):

        itemindex = np.where(y_val[:,i][j] == 1 )[0][0]

        temp_label[j] = itemindex

    y_test_indexes[i] = temp_label
    


In [666]:
# check for entire captchas correctly labeled

entire_correct = 0
entire_total =0 
for i in range(y_val.size()[1]):
    a = y_test_indexes[i] ==pred_labels[i]
    #print(a)

    if False not in a:

        entire_correct +=1

        entire_total+=1
        
    else:
        entire_total+=1

print(entire_correct/entire_total * 100)


15.887850467289718


In [674]:
# check how many digits correctly labeled

digits_correct = 0
total_digits = 0

for i in range(y_val.size()[1]):

    for j in range(5):

        if y_test_indexes[i][j] ==pred_labels[i][j]:
            digits_correct +=1
            total_digits +=1

        else:
            total_digits += 1 
    

print(digits_correct/total_digits * 100)

66.82242990654206
